In [ ]:
import pandas as pd
from pandasql import sqldf
from sqlalchemy import create_engine, text

import numpy as np
from numpy import dtype


In [4]:
import os
import sys
sys.path.append(os.path.abspath("../src"))

In [5]:
from  src import *

In [6]:
db = database_loader.DatabaseLoader(get_config('connection_string'))
DEBUG=False
SCHEMA='controls'

In [7]:
stats_df = db.read_table(table_name="team_weekly_stats", schema="controls")
stats_df.head()

,season,week,team,rush_yards,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,avg_rush_yards,rush_touchdowns,...,receiving_2pt_conversions,special_teams_tds,opponent,team_score,team_coach,opposing_team,opposing_score,opposing_coach,spread,ops_load_timestamp
0,2022,1,ARI,26,5.930769,10.000000,2.445889,10,2.600000,1,...,1,0,KC,21,Kliff Kingsbury,KC,44,Andy Reid,-23,2023-07-06 18:24:44.399127
1,2022,3,ARI,39,5.421026,7.692308,2.580545,13,3.000000,0,...,0,0,LA,12,Kliff Kingsbury,LA,20,Sean McVay,-8,2023-07-06 18:24:44.399127
2,2022,4,ARI,55,3.860545,40.000000,2.565267,15,3.666667,0,...,0,0,CAR,26,Kliff Kingsbury,CAR,16,Matt Rhule,10,2023-07-06 18:24:44.399127
3,2022,6,ARI,37,5.610541,6.666667,2.708000,15,2.466667,0,...,0,0,SEA,9,Kliff Kingsbury,SEA,19,Pete Carroll,-10,2023-07-06 18:24:44.399127
4,2022,7,ARI,92,2.619457,8.333333,2.810818,12,7.666667,1,...,1,0,NO,42,Kliff Kingsbury,NO,34,Dennis Allen,8,2023-07-06 18:24:44.399127


In [21]:
stats_df.drop(columns=['ops_load_timestamp'], inplace=True)

In [25]:
from collections import defaultdict

categories = defaultdict(lambda: [])

for column in stats_df.columns:
    n = len(stats_df[column].value_counts())
    if n < 25:
        categories[stats_df[column].dtype.name].append(column)

for k, v in categories.items():
    print("---")
    print(f"{k} columns : {v}")


---
int64 columns : ['season', 'week', 'rush_touchdowns', 'pass_touchdowns', 'interceptions', 'fumble', 'own_kickoff_recovery', 'safety', 'qb_hit', 'touchdown']
---
object columns : ['passing_tds', 'ps_interceptions', 'sacks', 'sack_fumbles', 'sack_fumbles_lost', 'passing_2pt_conversions', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'special_teams_tds']


In [27]:
numeric_columns_to_hide={'season','week', 'opposing_score', 'team_score', 'spread', 'interceptions'}

# all numeric columns and their values
numeric_columns = stats_df.select_dtypes(include='number').drop(columns=list( numeric_columns_to_hide) + categories['int64']) ## review whats left over
numeric_columns.columns

Index(['rush_yards', 'efficiency', 'percent_attempts_gte_eight_defenders',
       'avg_time_to_los', 'rush_attempts', 'avg_rush_yards',
       'player_jersey_number', 'avg_time_to_throw', 'avg_completed_air_yards',
       'avg_intended_air_yards', 'avg_air_yards_differential',
       'aggressiveness', 'avg_air_yards_to_sticks', 'attempts', 'pass_yards',
       'passer_rating', 'completions', 'completion_percentage', 'tackle',
       'avg_dakota'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
